In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential, load_model
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

In [2]:
# df = pd.read_csv("Processed_data.csv")
# df.drop("Unnamed: 0",inplace=True,axis=1)
# y = df['final_score']
# df.drop('final_score',inplace=True,axis=1)
# X=df
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
# X_train.shape

df = pd.read_csv("training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
temp = pd.read_csv("Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)
df['domain1_score']=temp['final_score']
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)


In [4]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)

In [5]:
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 vector_size=num_features, 
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)


In [10]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index_to_key)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model.wv[i])        
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[] 

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)
training_vectors.shape

C:\Users\Hasnat\AppData\Local\Temp\ipykernel_19504\2450526373.py:9: RuntimeWarning: invalid value encountered in divide
  vec = np.divide(vec,noOfWords)


(9083, 300)

In [11]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mse'])
    model.summary()
    return model

In [12]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 1, 300)            721200    
                                                                 
 lstm_3 (LSTM)               (None, 64)                93440     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


In [13]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=150)

Epoch 1/150
142/142 [==============================] - 4s 11ms/step - loss: 7.6908 - mse: 7.6908
Epoch 2/150
142/142 [==============================] - 2s 11ms/step - loss: 5.2102 - mse: 5.2102
Epoch 3/150
142/142 [==============================] - 2s 11ms/step - loss: 4.9624 - mse: 4.9624
Epoch 4/150
142/142 [==============================] - 2s 11ms/step - loss: 4.7243 - mse: 4.7243
Epoch 5/150
142/142 [==============================] - 2s 11ms/step - loss: 4.5830 - mse: 4.5830
Epoch 6/150
142/142 [==============================] - 1s 11ms/step - loss: 4.4746 - mse: 4.4746
Epoch 7/150
142/142 [==============================] - 1s 11ms/step - loss: 4.4217 - mse: 4.4217
Epoch 8/150
142/142 [==============================] - 2s 11ms/step - loss: 4.3422 - mse: 4.3422
Epoch 9/150
142/142 [==============================] - 2s 11ms/step - loss: 4.2866 - mse: 4.2866
Epoch 10/150
142/142 [==============================] - 2s 11ms/step - loss: 4.1978 - mse: 4.1978
Epoch 11/150
142/142 [=======

In [14]:
lstm_model.save('lstm_model.h5')
lstm_model = load_model("lstm_model.h5")
y_pred = lstm_model.predict(testing_vectors)

122/122 [==============================] - 1s 2ms/step


In [ ]:
# for l in range(5):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=l, shuffle=True)
#     train_e = X_train['essay'].tolist()
#     test_e = X_test['essay'].tolist()
#     train_sents=[]
#     test_sents=[]
#     for i in train_e:
#         train_sents+=essay2word(i)

#     for i in test_e:
#         test_sents+=essay2word(i)
    
#     num_features = 300 
#     min_word_count = 40
#     num_workers = 8
#     context = 10
#     downsampling = 1e-3

#     model = Word2Vec(train_sents, 
#                     workers=num_workers, 
#                     vector_size=num_features, 
#                     min_count = min_word_count, 
#                     window = context, 
#                     sample = downsampling)
#     model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

#     clean_train=[]
#     for i in train_e:
#         clean_train.append(sent2word(i))
#     training_vectors = getVecs(clean_train, model, num_features)

#     clean_test=[] 

#     for i in test_e:
#         clean_test.append(sent2word(i))
#     testing_vectors = getVecs(clean_test, model, num_features)
    
#     training_vectors = np.array(training_vectors)
#     testing_vectors = np.array(testing_vectors)

#     training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
#     testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
#     lstm_model = get_model()
#     lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=10)

#     if l == 4:
#         lstm_model.save_weights('lstm_model.h5')

#     y_pred = lstm_model.predict(testing_vectors)
#     y_pred = np.around(y_pred)

#     y_test_list = y_test.tolist()
#     y_pred_list = []

#     for i in range(len(y_pred)):
#         if str(y_pred[i][0]) == 'nan':
#             y_pred[i][0] = 0    
#         y_pred_list.append(round(y_pred[i][0]))
#     result = cohen_kappa_score(y_test_list, y_pred_list,weights='quadratic')

#     print("Kappa Score: {}".format(result))

In [16]:
from sklearn.metrics import cohen_kappa_score

y_test_list = y_test.tolist()
y_pred_list = []

for i in range(len(y_pred)):
    if str(y_pred[i][0]) == 'nan':
        y_pred[i][0] = 0    
    y_pred_list.append(round(y_pred[i][0]))
kappa = cohen_kappa_score(y_test_list, y_pred_list,weights='quadratic')
kappa


0.6921650568287518